In [ ]:
#Working with DF, Spark SQL and UDF

In [ ]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=a3622277a5100469e467d43d0e77f6c58340555194055aadd6dacc45912600a0
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [ ]:
import pyspark
from pyspark.sql import SparkSession
spark=SparkSession.builder.getOrCreate()

In [ ]:
df=spark.read.format("CSV").load("CompleteDataset.csv",inferSchema=True, header=True)
df.show()

+---+-----------------+---+--------------------+-----------+--------------------+-------+---------+-------------------+--------------------+------+-----+-------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+---------+-----------+----------+--------------+-----------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+-------+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+-------------------+----+----+----+----+----+----+----+----+----+----+----+
|_c0|             Name|Age|               Photo|Nationality|                Flag|Overall|Potential|               Club|           Club Logo| Value| Wage|Special|Acceleration|Aggression|Agility|Balance|Ball control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|GK diving|GK handling|

In [ ]:
df.rdd.getNumPartitions()

2

In [ ]:
df_change=df.withColumnRenamed("Club Logo","Club_logo")\
.withColumnRenamed("Flag","National Flag")

In [ ]:
df_change.show()

+---+-----------------+---+--------------------+-----------+--------------------+-------+---------+-------------------+--------------------+------+-----+-------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+---------+-----------+----------+--------------+-----------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+-------+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+-------------------+----+----+----+----+----+----+----+----+----+----+----+
|_c0|             Name|Age|               Photo|Nationality|       National Flag|Overall|Potential|               Club|           Club_logo| Value| Wage|Special|Acceleration|Aggression|Agility|Balance|Ball control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|GK diving|GK handling|

In [ ]:
df_change.na.fill({"CAM":85,"CB":50}).show()

+---+-----------------+---+--------------------+-----------+--------------------+-------+---------+-------------------+--------------------+------+-----+-------+------------+----------+-------+-------+------------+---------+--------+-----+---------+---------+------------------+---------+-----------+----------+--------------+-----------+----------------+-------------+-------+------------+----------+-------+---------+-----------+---------+-------------+----------+--------------+------------+-------+---------------+--------+------+-------+----+----+----+----+----+------+----+----+----+----+----+----+----+----+----+----+-------------------+----+----+----+----+----+----+----+----+----+----+----+
|_c0|             Name|Age|               Photo|Nationality|       National Flag|Overall|Potential|               Club|           Club_logo| Value| Wage|Special|Acceleration|Aggression|Agility|Balance|Ball control|Composure|Crossing|Curve|Dribbling|Finishing|Free kick accuracy|GK diving|GK handling|

In [ ]:
df_change.select("Name","Nationality","Overall","Club","Wage").filter(df_change["Overall"]>85).show()

+-----------------+-----------+-------+-------------------+-----+
|             Name|Nationality|Overall|               Club| Wage|
+-----------------+-----------+-------+-------------------+-----+
|Cristiano Ronaldo|   Portugal|     94|     Real Madrid CF|€565K|
|         L. Messi|  Argentina|     93|       FC Barcelona|€565K|
|           Neymar|     Brazil|     92|Paris Saint-Germain|€280K|
|        L. Suárez|    Uruguay|     92|       FC Barcelona|€510K|
|         M. Neuer|    Germany|     92|   FC Bayern Munich|€230K|
|   R. Lewandowski|     Poland|     91|   FC Bayern Munich|€355K|
|           De Gea|      Spain|     90|  Manchester United|€215K|
|        E. Hazard|    Belgium|     90|            Chelsea|€295K|
|         T. Kroos|    Germany|     90|     Real Madrid CF|€340K|
|       G. Higuaín|  Argentina|     90|           Juventus|€275K|
|     Sergio Ramos|      Spain|     90|     Real Madrid CF|€310K|
|     K. De Bruyne|    Belgium|     89|    Manchester City|€285K|
|      T. 

In [ ]:
df_change.select("Name","Nationality","Overall","Club","Wage").filter(df_change["Overall"]>85).groupBy("Nationality").count().show()

+--------------+-----+
|   Nationality|count|
+--------------+-----+
|        Sweden|    1|
|       Germany|    7|
|        France|    6|
|        Greece|    1|
|      Slovakia|    1|
|     Argentina|    4|
|         Wales|    1|
|       Belgium|    7|
|         Chile|    2|
|       Croatia|    2|
|         Gabon|    1|
|         Italy|    4|
|         Spain|   11|
|       Denmark|    1|
|       Uruguay|    3|
|Czech Republic|    1|
|        Brazil|    7|
|      Slovenia|    2|
|       England|    1|
|        Poland|    1|
+--------------+-----+
only showing top 20 rows



In [ ]:
#Spark SQL and UDF
df_change.createOrReplaceTempView("Football_table")

In [ ]:
query="""select Name,Nationality,Overall,Club,Wage
          from Football_table
          where Overall>85"""
result=spark.sql(query)
result.show()

+-----------------+-----------+-------+-------------------+-----+
|             Name|Nationality|Overall|               Club| Wage|
+-----------------+-----------+-------+-------------------+-----+
|Cristiano Ronaldo|   Portugal|     94|     Real Madrid CF|€565K|
|         L. Messi|  Argentina|     93|       FC Barcelona|€565K|
|           Neymar|     Brazil|     92|Paris Saint-Germain|€280K|
|        L. Suárez|    Uruguay|     92|       FC Barcelona|€510K|
|         M. Neuer|    Germany|     92|   FC Bayern Munich|€230K|
|   R. Lewandowski|     Poland|     91|   FC Bayern Munich|€355K|
|           De Gea|      Spain|     90|  Manchester United|€215K|
|        E. Hazard|    Belgium|     90|            Chelsea|€295K|
|         T. Kroos|    Germany|     90|     Real Madrid CF|€340K|
|       G. Higuaín|  Argentina|     90|           Juventus|€275K|
|     Sergio Ramos|      Spain|     90|     Real Madrid CF|€310K|
|     K. De Bruyne|    Belgium|     89|    Manchester City|€285K|
|      T. 

In [ ]:
#UDF

def changeName(name):
  if len(name)> 7:
    return name.upper()
  else:
    return name

spark.udf.register("UPP",changeName)

<function __main__.changeName(name)>

In [ ]:
query2="""select UPP(name) as Name, Nationality, Overall
          from Football_table
          where overall>90
          order by name asc"""

result2=spark.sql(query2)
result2.show()

+-----------------+-----------+-------+
|             Name|Nationality|Overall|
+-----------------+-----------+-------+
|CRISTIANO RONALDO|   Portugal|     94|
|         L. MESSI|  Argentina|     93|
|        L. SUÁREZ|    Uruguay|     92|
|         M. NEUER|    Germany|     92|
|           Neymar|     Brazil|     92|
|   R. LEWANDOWSKI|     Poland|     91|
+-----------------+-----------+-------+

